In [1]:
from datasets import load_dataset

# Load dataset
print("Loading dataset...")
train_ds = load_dataset("cnamuangtoun/resume-job-description-fit", split="train")
test_ds = load_dataset("cnamuangtoun/resume-job-description-fit", split="test")

print(f"\n📊 Dataset sizes:")
print(f"  Train: {len(train_ds):,}")
print(f"  Test:  {len(test_ds):,}")

/opt/homebrew/Caskroom/miniconda/base/envs/ml-env/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


Loading dataset...


train.csv:   0%|          | 0.00/53.4M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/15.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6241 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1759 [00:00<?, ? examples/s]


📊 Dataset sizes:
  Train: 6,241
  Test:  1,759


In [26]:
import re
from collections import Counter

# ==================== 學校分類 (School Classification) ====================
# Top schools 列表 - 使用更精確的 patterns
TOP_SCHOOLS_PATTERNS = {
    # Ivy League - 完整名稱 + 常見縮寫
    'harvard': r'\bharvard\s+university\b|\bharvard\b',
    'yale': r'\byale\s+university\b|\byale\b',
    'princeton': r'\bprinceton\s+university\b|\bprinceton\b',
    'columbia': r'\bcolumbia\s+university\b',
    'upenn': r'\buniversity\s+of\s+pennsylvania\b|\bupenn\b|\bu\s+penn\b',
    'dartmouth': r'\bdartmouth\s+(college|university)\b|\bdartmouth\b',
    'brown': r'\bbrown\s+university\b',
    'cornell': r'\bcornell\s+university\b|\bcornell\b',
    
    # Top US Universities
    'stanford': r'\bstanford\s+university\b|\bstanford\b',
    'mit': r'\bmassachusetts\s+institute\s+of\s+technology\b|\b(?:mit)\b(?!\w)',
    'caltech': r'\bcalifornia\s+institute\s+of\s+technology\b|\bcaltech\b',
    'uchicago': r'\buniversity\s+of\s+chicago\b|\bchicago\s+university\b',
    'duke': r'\bduke\s+university\b|\bduke\b',
    'northwestern': r'\bnorthwestern\s+university\b|\bnorthwestern\b',
    'johns_hopkins': r'\bjohns\s+hopkins\s+university\b|\bjohns\s+hopkins\b',
    'vanderbilt': r'\bvanderbilt\s+university\b|\bvanderbilt\b',
    'rice': r'\brice\s+university\b',
    'notre_dame': r'\buniversity\s+of\s+notre\s+dame\b|\bnotre\s+dame\b',
    
    # UC System
    'berkeley': r'\buc\s+berkeley\b|\buniversity\s+of\s+california,?\s+berkeley\b|\bberkeley\s+university\b',
    'ucla': r'\bucla\b|\buniversity\s+of\s+california,?\s+los\s+angeles\b',
    'ucsd': r'\bucsd\b|\buniversity\s+of\s+california,?\s+san\s+diego\b',
    'usc': r'\busc\b(?=\s+(?:university|school|college))|\buniversity\s+of\s+southern\s+california\b',
    
    # Other Top US
    'carnegie_mellon': r'\bcarnegie\s+mellon\s+university\b|\bcarnegie\s+mellon\b|\bcmu\b',
    'georgetown': r'\bgeorgetown\s+university\b|\bgeorgetown\b',
    'michigan': r'\buniversity\s+of\s+michigan\b|\bumich\b|\bu\s+michigan\b',
    'virginia': r'\buniversity\s+of\s+virginia\b|\buva\b',
    
    # UK Universities
    'oxford': r'\buniversity\s+of\s+oxford\b|\boxford\s+university\b',
    'cambridge': r'\buniversity\s+of\s+cambridge\b|\bcambridge\s+university\b',
    'imperial': r'\bimperial\s+college\s+london\b|\bimperial\s+college\b',
    'lse': r'\blondon\s+school\s+of\s+economics\b|\blse\b(?!\w)',
    'ucl': r'\buniversity\s+college\s+london\b|\bucl\b(?!\w)',
    'edinburgh': r'\buniversity\s+of\s+edinburgh\b|\bedinburgh\s+university\b',
    
    # Canadian
    'toronto': r'\buniversity\s+of\s+toronto\b|\bu\s+of\s+t\b',
    'mcgill': r'\bmcgill\s+university\b|\bmcgill\b',
    'ubc': r'\buniversity\s+of\s+british\s+columbia\b|\bubc\b(?!\w)',
    
    # Asian
    'national_taiwan_university': r'\bnational\s+taiwan\s+university\b',
    'ntu_singapore': r'\bnanyang\s+technological\s+university\b|\bntu\s+singapore\b',
    'nus': r'\bnational\s+university\s+of\s+singapore\b|\bnus\b(?!\w)',
    'peking': r'\bpeking\s+university\b|\bpku\b',
    'tsinghua': r'\btsinghua\s+university\b',
    'tokyo': r'\buniversity\s+of\s+tokyo\b|\btokyo\s+university\b',
    
    # Australian
    'melbourne': r'\buniversity\s+of\s+melbourne\b|\bmelbourne\s+university\b',
    'sydney': r'\buniversity\s+of\s+sydney\b|\bsydney\s+university\b',
    
    # Public Ivies
    'unc': r'\buniversity\s+of\s+north\s+carolina\b|\bunc\s+chapel\s+hill\b|\bunc\b(?=\s+(?:at|chapel))',
}

def classify_school(text):
    """
    分類學校為 top school 或 non-top school (使用嚴格的 word boundary)
    Returns: dict with school classification
    """
    if not text or not isinstance(text, str):
        return {
            'has_school': False,
            'is_top_school': False,
            'school_category': 'unknown',
            'matched_schools': []
        }
    
    text_lower = text.lower()
    
    # 尋找提到的 top schools (使用正則表達式精確匹配)
    matched_schools = []
    for school_name, pattern in TOP_SCHOOLS_PATTERNS.items():
        if re.search(pattern, text_lower):
            matched_schools.append(school_name)
    
    # 檢查是否有「university」或「college」字樣
    has_school_mention = bool(re.search(r'\b(university|college|institute|school)\b', text_lower))
    
    if matched_schools:
        school_category = 'top_school'
        is_top_school = True
    elif has_school_mention:
        school_category = 'non_top_school'
        is_top_school = False
    else:
        school_category = 'no_school_mentioned'
        is_top_school = False
    
    return {
        'has_school': has_school_mention or len(matched_schools) > 0,
        'is_top_school': is_top_school,
        'school_category': school_category,
        'matched_schools': matched_schools
    }


# ==================== 應用到資料集 ====================
print("🔍 Extracting school classifications...")
print("\n" + "="*60)

# 處理訓練集
print("\n📝 Processing training set...")
train_school_stats = []

for idx, example in enumerate(train_ds):
    resume = example.get('resume_text', '')
    
    # 學校分類
    school_info = classify_school(resume)
    train_school_stats.append(school_info)
    
    if (idx + 1) % 1000 == 0:
        print(f"  Processed {idx + 1:,} examples...")

# 處理測試集
print("\n📝 Processing test set...")
test_school_stats = []

for idx, example in enumerate(test_ds):
    resume = example.get('resume_text', '')
    
    # 學校分類
    school_info = classify_school(resume)
    test_school_stats.append(school_info)
    
    if (idx + 1) % 1000 == 0:
        print(f"  Processed {idx + 1:,} examples...")

print("\n✅ Processing complete!")
print("="*60)

🔍 Extracting school classifications...


📝 Processing training set...
  Processed 1,000 examples...
  Processed 1,000 examples...
  Processed 2,000 examples...
  Processed 2,000 examples...
  Processed 3,000 examples...
  Processed 3,000 examples...
  Processed 4,000 examples...
  Processed 4,000 examples...
  Processed 5,000 examples...
  Processed 5,000 examples...
  Processed 6,000 examples...
  Processed 6,000 examples...

📝 Processing test set...

📝 Processing test set...
  Processed 1,000 examples...
  Processed 1,000 examples...

✅ Processing complete!

✅ Processing complete!


In [27]:
# ==================== 統計分析 ====================
print("\n📊 SCHOOL CLASSIFICATION STATISTICS")
print("="*60)

# 訓練集學校統計
train_with_school = sum(1 for s in train_school_stats if s['has_school'])
train_top_school = sum(1 for s in train_school_stats if s['is_top_school'])
train_non_top = sum(1 for s in train_school_stats if s['school_category'] == 'non_top_school')

print(f"\n📚 Training Set (n={len(train_ds):,}):")
print(f"  With school mention:   {train_with_school:,} ({train_with_school/len(train_ds)*100:.2f}%)")
print(f"    └─ Top schools:       {train_top_school:,} ({train_top_school/len(train_ds)*100:.2f}%)")
print(f"    └─ Non-top schools:   {train_non_top:,} ({train_non_top/len(train_ds)*100:.2f}%)")

# 測試集學校統計
test_with_school = sum(1 for s in test_school_stats if s['has_school'])
test_top_school = sum(1 for s in test_school_stats if s['is_top_school'])
test_non_top = sum(1 for s in test_school_stats if s['school_category'] == 'non_top_school')

print(f"\n🧪 Test Set (n={len(test_ds):,}):")
print(f"  With school mention:   {test_with_school:,} ({test_with_school/len(test_ds)*100:.2f}%)")
print(f"    └─ Top schools:       {test_top_school:,} ({test_top_school/len(test_ds)*100:.2f}%)")
print(f"    └─ Non-top schools:   {test_non_top:,} ({test_non_top/len(test_ds)*100:.2f}%)")

# 最常見的 top schools
all_matched = []
for s in train_school_stats + test_school_stats:
    all_matched.extend(s['matched_schools'])

if all_matched:
    top_schools_counter = Counter(all_matched)
    print(f"\n🎓 Top 10 Most Mentioned Elite Schools:")
    for school, count in top_schools_counter.most_common(10):
        print(f"  {school:.<30} {count:>5} mentions")

print("\n" + "="*60)


📊 SCHOOL CLASSIFICATION STATISTICS

📚 Training Set (n=6,241):
  With school mention:   5,212 (83.51%)
    └─ Top schools:       252 (4.04%)
    └─ Non-top schools:   4,960 (79.47%)

🧪 Test Set (n=1,759):
  With school mention:   1,495 (84.99%)
    └─ Top schools:       73 (4.15%)
    └─ Non-top schools:   1,422 (80.84%)

🎓 Top 10 Most Mentioned Elite Schools:
  berkeley......................    66 mentions
  uchicago......................    44 mentions
  peking........................    43 mentions
  ucla..........................    34 mentions
  princeton.....................    23 mentions
  dartmouth.....................    20 mentions
  johns_hopkins.................    18 mentions
  northwestern..................    15 mentions
  michigan......................    14 mentions
  upenn.........................    14 mentions



In [28]:
# ==================== 示例展示 ====================
print("\n📋 SAMPLE EXAMPLES")
print("="*60)

# 找出有 top school 的範例
print("\n🎓 Examples with Top Schools:\n")
school_examples_shown = 0
for idx, (example, school_info) in enumerate(zip(train_ds, train_school_stats)):
    if school_info['is_top_school'] and school_examples_shown < 5:
        resume = example.get('resume_text', '')
        print(f"Example {school_examples_shown + 1}:")
        print(f"  School Category: {school_info['school_category']}")
        print(f"  Matched Schools: {school_info['matched_schools']}")
        print(f"  Resume snippet: {resume[:250]}...")
        print()
        school_examples_shown += 1
    if school_examples_shown >= 5:
        break

print("="*60)


📋 SAMPLE EXAMPLES

🎓 Examples with Top Schools:

Example 1:
  School Category: top_school
  Matched Schools: ['yale']
  Resume snippet: ExperienceData, Research & Marketing Consultant,01/2019-01/2019Pfizer–Columbus,OH,Https://rpinsights.com, https://rpmarketingco.com.Designing membership surveys and managing CRM databases for non-profit organization.Helping businesses with branding, ...

Example 2:
  School Category: top_school
  Matched Schools: ['uchicago']
  Resume snippet: QUALIFICATIONS•   Over two years' experience of data analysis
at Loyola Chicago University; providing statistical analysis plan and
generating report, tables, graphs;
proficient in R and SAS.•   Proficient in Microsoft Office, Word, Excel
and Power P...

Example 3:
  School Category: top_school
  Matched Schools: ['ucla']
  Resume snippet: SummaryDedi100 Montgomery St. 10th Floorted Health 100 Montgomery St. 10th Floorre Administrator at improving team productivity, policies, clini100 Montgomery St. 10th Floorl 

In [29]:
# ==================== 詳細驗證分析 ====================
print("\n🔬 DETAILED VALIDATION")
print("="*60)

# 學校分類驗證 - 查看一些實際匹配
print("\n🎓 School Detection Validation:\n")
school_with_context = []
for idx, (example, school_info) in enumerate(zip(train_ds, train_school_stats)):
    if school_info['is_top_school']:
        resume = example.get('resume_text', '')
        for school in school_info['matched_schools']:
            # 找出學校名稱在文本中的上下文
            text_lower = resume.lower()
            for match in re.finditer(r'\b\w*(?:university|college|school|institute)\w*\b', text_lower):
                context = resume[max(0, match.start()-30):min(len(resume), match.end()+30)]
                if any(s.replace('_', ' ') in context.lower() for s in school_info['matched_schools']):
                    school_with_context.append({
                        'school': school,
                        'context': context.strip()
                    })
                    break
            if len(school_with_context) >= 15:
                break
    if len(school_with_context) >= 15:
        break

print("Sample matches with context:")
for i, item in enumerate(school_with_context[:10], 1):
    print(f"\n{i}. School: {item['school']}")
    print(f"   Context: ...{item['context']}...")

print("\n" + "="*60)


🔬 DETAILED VALIDATION

🎓 School Detection Validation:

Sample matches with context:

1. School: yale
   Context: ...sophy (PhD):,Expected in-Yale University-New Haven,CTGPA:Status-Bachel...

2. School: ucla
   Context: ...y College, Glendale Community College, Northridge University, UCLA,...

3. School: dartmouth
   Context: ...project components.Dartmouth College-DATA MANAGER/EPIDEMIOLOGISTHa...

4. School: michigan
   Context: ...project components.Dartmouth College-DATA MANAGER/EPIDEMIOLOGISTHa...

5. School: unc
   Context: ...project components.Dartmouth College-DATA MANAGER/EPIDEMIOLOGISTHa...

6. School: harvard
   Context: ...cted in2021toHarvard Business School (HBS)-,GPA:Admitted to the co...

7. School: berkeley
   Context: ...ment,Expected in2019-Berkeley University of California-San Francisco,C...

8. School: dartmouth
   Context: ...s-section scan data.Dartmouth College-Product Engineering Intern,,-...

9. School: yale
   Context: ...sophy (PhD):,Expected in-Yale Univers

In [31]:
# ==================== 將特徵加入資料集 ====================
print("\n💾 Adding features to datasets...")

# 移除舊的欄位（如果存在）
cols_to_remove = ['has_gender_indicator', 'inferred_gender', 'male_count', 'female_count', 
                  'is_top_school', 'school_category', 'matched_schools']
for col in cols_to_remove:
    if col in train_ds.column_names:
        train_ds = train_ds.remove_columns(col)
    if col in test_ds.column_names:
        test_ds = test_ds.remove_columns(col)

# 為訓練集添加特徵
train_ds = train_ds.add_column('is_top_school', [s['is_top_school'] for s in train_school_stats])
train_ds = train_ds.add_column('school_category', [s['school_category'] for s in train_school_stats])

# 為測試集添加特徵
test_ds = test_ds.add_column('is_top_school', [s['is_top_school'] for s in test_school_stats])
test_ds = test_ds.add_column('school_category', [s['school_category'] for s in test_school_stats])

print("✅ Features added successfully!")
print(f"\n📊 Updated dataset columns:")
print(f"  {list(train_ds.features.keys())}")
print("\n" + "="*60)


💾 Adding features to datasets...
✅ Features added successfully!

📊 Updated dataset columns:
  ['resume_text', 'job_description_text', 'label', 'is_top_school', 'school_category']



In [32]:
# ==================== 保存處理後的資料 (可選) ====================
# 如果需要保存處理後的資料集，取消下方註解

# from datasets import DatasetDict
# 
# dataset_dict = DatasetDict({
#     'train': train_ds,
#     'test': test_ds
# })
# 
# # 保存到本地
# dataset_dict.save_to_disk('./processed_resume_dataset')
# print("✅ Dataset saved to ./processed_resume_dataset")
# 
# # 或推送到 Hugging Face Hub (需要先登入)
# # dataset_dict.push_to_hub("your-username/resume-job-fit-with-fairness-features")

print("\n✨ Data preparation complete!")
print(f"\n📦 Datasets ready for fairness analysis:")
print(f"  - train_ds: {len(train_ds):,} examples with {len(train_ds.column_names)} features")
print(f"  - test_ds:  {len(test_ds):,} examples with {len(test_ds.column_names)} features")


✨ Data preparation complete!

📦 Datasets ready for fairness analysis:
  - train_ds: 6,241 examples with 5 features
  - test_ds:  1,759 examples with 5 features


In [5]:
# ==================== 檢查資料格式 ====================
print("🔍 Inspecting actual data format...\n")
print("="*60)

# 查看第一筆資料的結構
print("📋 First example structure:")
print(f"Keys: {list(train_ds[0].keys())}\n")

# 查看前3筆履歷的完整內容
for i in range(min(3, len(train_ds))):
    print(f"\n{'='*60}")
    print(f"Example {i+1}:")
    print(f"{'='*60}")
    example = train_ds[i]
    
    # 顯示所有欄位
    for key, value in example.items():
        if isinstance(value, str):
            preview = value[:300] + "..." if len(value) > 300 else value
            print(f"\n[{key}]:")
            print(f"{preview}")
        else:
            print(f"\n[{key}]: {value}")
    print()

print("="*60)

🔍 Inspecting actual data format...

📋 First example structure:
Keys: ['resume_text', 'job_description_text', 'label']


Example 1:

[resume_text]:
SummaryHighly motivated Sales Associate with extensive customer service and sales experience. Outgoing sales professional with track record of driving increased sales, improving buying experience and elevating company profile with target market.
Highlights-Soft Skills: Public Speaking, Public Relati...

[job_description_text]:
Net2Source Inc. is an award-winning total workforce solutions company recognized by Staffing Industry Analysts for our accelerated growth of 300% in the last 3 years with over 5500+ employees globally, with over 30+ locations in the US and global operations in 32 countries. We believe in providing s...

[label]:
No Fit


Example 2:

[resume_text]:
Professional SummaryCurrently working with Caterpillar as an contract employee Active in several NPI and CPI projects Provide technical for CAT facilities worldwide 24-hours 

In [33]:
# ==================== 保存處理後的資料集 ====================
from datasets import DatasetDict
import os

print("💾 Saving processed datasets...")
print("="*60)

# 建立輸出目錄
output_dir = './processed_resume_dataset'
os.makedirs(output_dir, exist_ok=True)

# 建立 DatasetDict
dataset_dict = DatasetDict({
    'train': train_ds,
    'test': test_ds
})

# 保存到本地
dataset_dict.save_to_disk(output_dir)
print(f"✅ Dataset saved to: {output_dir}")

# 顯示資料集資訊
print(f"\n📊 Saved Dataset Information:")
print(f"  📁 Location: {output_dir}")
print(f"  📝 Train: {len(train_ds):,} examples")
print(f"  🧪 Test:  {len(test_ds):,} examples")
print(f"  🏷️  Features: {list(train_ds.features.keys())}")
print(f"\n🎯 School Classification Features:")
print(f"  - is_top_school: Boolean (是否為頂尖學校)")
print(f"  - school_category: String (top_school/non_top_school/no_school_mentioned)")

print("\n" + "="*60)
print("✨ All done! Dataset is ready for use.")
print("="*60)

💾 Saving processed datasets...


Saving the dataset (0/1 shards):   0%|          | 0/6241 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1759 [00:00<?, ? examples/s]

✅ Dataset saved to: ./processed_resume_dataset

📊 Saved Dataset Information:
  📁 Location: ./processed_resume_dataset
  📝 Train: 6,241 examples
  🧪 Test:  1,759 examples
  🏷️  Features: ['resume_text', 'job_description_text', 'label', 'is_top_school', 'school_category']

🎯 School Classification Features:
  - is_top_school: Boolean (是否為頂尖學校)
  - school_category: String (top_school/non_top_school/no_school_mentioned)

✨ All done! Dataset is ready for use.


## 📚 如何使用保存的資料集

### 載入資料集

```python
from datasets import load_from_disk

# 載入資料集
dataset = load_from_disk('./processed_resume_dataset')

# 訪問訓練集和測試集
train_data = dataset['train']
test_data = dataset['test']
```

### 資料集欄位說明

| 欄位名稱 | 類型 | 說明 |
|---------|------|------|
| `resume_text` | String | 履歷文本內容 |
| `job_description_text` | String | 職位描述文本 |
| `label` | String | 匹配標籤 (Fit/No Fit) |
| `is_top_school` | Boolean | 是否提及頂尖學校 |
| `school_category` | String | 學校類別 (top_school/non_top_school/no_school_mentioned) |

### 範例使用

```python
# 查看第一筆資料
print(train_data[0])

# 篩選包含頂尖學校的履歷
top_school_resumes = train_data.filter(lambda x: x['is_top_school'])
print(f"Top school resumes: {len(top_school_resumes)}")

# 按學校類別分組
from collections import Counter
school_dist = Counter(train_data['school_category'])
print(school_dist)
```

In [34]:
# ==================== 驗證資料集載入 ====================
print("🔍 Verifying saved dataset...")
print("="*60)

from datasets import load_from_disk

# 重新載入資料集進行驗證
loaded_dataset = load_from_disk('./processed_resume_dataset')

print(f"\n✅ Dataset loaded successfully!")
print(f"\n📊 Dataset Structure:")
print(f"  Splits: {list(loaded_dataset.keys())}")
print(f"  Train size: {len(loaded_dataset['train']):,}")
print(f"  Test size: {len(loaded_dataset['test']):,}")

print(f"\n🏷️  Features:")
for feature_name, feature_type in loaded_dataset['train'].features.items():
    print(f"  - {feature_name}: {feature_type}")

print(f"\n📝 Sample Data (First Example):")
sample = loaded_dataset['train'][0]
print(f"  Resume (first 150 chars): {sample['resume_text'][:150]}...")
print(f"  Label: {sample['label']}")
print(f"  Is Top School: {sample['is_top_school']}")
print(f"  School Category: {sample['school_category']}")

print(f"\n📈 School Distribution:")
from collections import Counter
train_schools = Counter(loaded_dataset['train']['school_category'])
for category, count in train_schools.items():
    pct = count / len(loaded_dataset['train']) * 100
    print(f"  {category:.<25} {count:>5} ({pct:>5.2f}%)")

print("\n" + "="*60)
print("✅ Verification complete - Dataset is ready to use!")
print("="*60)

🔍 Verifying saved dataset...

✅ Dataset loaded successfully!

📊 Dataset Structure:
  Splits: ['train', 'test']
  Train size: 6,241
  Test size: 1,759

🏷️  Features:
  - resume_text: Value('string')
  - job_description_text: Value('string')
  - label: Value('string')
  - is_top_school: Value('bool')
  - school_category: Value('string')

📝 Sample Data (First Example):
  Resume (first 150 chars): SummaryHighly motivated Sales Associate with extensive customer service and sales experience. Outgoing sales professional with track record of driving...
  Label: No Fit
  Is Top School: False
  School Category: non_top_school

📈 School Distribution:
  non_top_school...........  4960 (79.47%)
  top_school...............   252 ( 4.04%)
  no_school_mentioned......  1029 (16.49%)

✅ Verification complete - Dataset is ready to use!
